## Import the Libraries

In [1]:
!pip install timm torch torchvision

     ---------------------------------------- 0.0/59.9 kB ? eta -:--:--
     ------ --------------------------------- 10.2/59.9 kB ? eta -:--:--
     -------------------------------- ----- 51.2/59.9 kB 435.7 kB/s eta 0:00:01
     -------------------------------------- 59.9/59.9 kB 455.3 kB/s eta 0:00:00
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.5 MB 5.3 MB/s eta 0:00:01
   --------- ------------------------------ 0.6/2.5 MB 7.6 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.5 MB 8.6 MB/s eta 0:00:01
   --------------- ------------------------ 1.0/2.5 MB 8.6 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 9.4 MB/s eta 0:00:01
   ---------------------------------------- 2.5/2.5 MB 9.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/216.1 MB ? eta -:--:--
   ---------------------


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import timm
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

import torch.optim as optim
from torch.nn import CrossEntropyLoss
from tqdm import tqdm

In [3]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder

label_file = pd.read_csv('Dataset/labels_file.csv')
label_file_clean = label_file.dropna(subset=['choice'])

# Extract and clean
image_paths = label_file_clean['image'].apply(os.path.basename).tolist()
labels = label_file_clean['choice'].tolist()

# Encode labels (e.g., casual → 0, streetwear → 1, etc.)
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)  # numpy array of integers


FileNotFoundError: [Errno 2] No such file or directory: 'Dataset/labels_file.csv'

In [ ]:
from PIL import Image
from torch.utils.data import Dataset

class FashionCSVDataset(Dataset):
    def __init__(self, image_paths, labels, image_dir, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_paths[idx])
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label


In [ ]:
from sklearn.model_selection import train_test_split
from torchvision import transforms
from torch.utils.data import DataLoader

# Split first
train_paths, val_paths, train_labels, val_labels = train_test_split(
    image_paths, encoded_labels, test_size=0.2, stratify=encoded_labels, random_state=42
)

# Transforms
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3),
])

# Dataset instances (only once — your previous code had duplicates)
image_dir = "Dataset/images"

train_dataset = FashionCSVDataset(train_paths, train_labels, image_dir, train_transform)
val_dataset   = FashionCSVDataset(val_paths, val_labels, image_dir, val_transform)

# Loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32)


In [ ]:
import timm
import torch.nn as nn
import torch

# Load EfficientNet with pretrained ImageNet weights
model = timm.create_model('efficientnet_b0', pretrained=True)

# Replace classifier head
num_classes = len(set(encoded_labels))  # e.g., 5 styles
model.classifier = nn.Sequential(
    nn.Dropout(0.3),
    nn.Linear(model.classifier.in_features, num_classes)
)

# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
criterion = CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from tqdm import tqdm
from sklearn.metrics import classification_report

num_epochs = 20

for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_acc = 100 * correct / total
    print(f"Train Loss: {train_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}%")

    # --- Validation ---
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_acc = 100 * correct / total
    print(f"Val Loss: {val_loss/len(val_loader):.4f} | Val Acc: {val_acc:.2f}%\n")

    # ➕ Add this for classification report
    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    print(classification_report(y_true, y_pred, target_names=label_encoder.classes_))



Epoch 1/20: 100%|██████████| 15/15 [05:24<00:00, 21.65s/it]


Train Loss: 1.5966 | Train Acc: 22.80%
Val Loss: 1.5426 | Val Acc: 26.50%

              precision    recall  f1-score   support

      casual       0.28      0.33      0.30        30
      formal       0.24      0.25      0.24        24
      sporty       0.00      0.00      0.00        10
  streetwear       0.32      0.35      0.33        37
     vintage       0.14      0.12      0.13        16

    accuracy                           0.26       117
   macro avg       0.20      0.21      0.20       117
weighted avg       0.24      0.26      0.25       117



Epoch 2/20: 100%|██████████| 15/15 [04:43<00:00, 18.90s/it]


Train Loss: 1.4334 | Train Acc: 44.95%
Val Loss: 1.4949 | Val Acc: 39.32%

              precision    recall  f1-score   support

      casual       0.38      0.37      0.37        30
      formal       0.50      0.29      0.37        24
      sporty       0.00      0.00      0.00        10
  streetwear       0.39      0.76      0.52        37
     vintage       0.00      0.00      0.00        16

    accuracy                           0.39       117
   macro avg       0.25      0.28      0.25       117
weighted avg       0.32      0.39      0.34       117



Epoch 3/20: 100%|██████████| 15/15 [04:45<00:00, 19.04s/it]


...

Train Loss: 0.0393 | Train Acc: 100.00%
Val Loss: 1.8683 | Val Acc: 41.88%

              precision    recall  f1-score   support

      casual       0.43      0.40      0.41        30
      formal       0.52      0.58      0.55        24
      sporty       0.25      0.20      0.22        10
  streetwear       0.53      0.51      0.52        37
     vintage       0.11      0.12      0.12        16

    accuracy                           0.42       117
   macro avg       0.37      0.36      0.36       117
weighted avg       0.42      0.42      0.42       117



Epoch 19/20: 100%|██████████| 15/15 [02:59<00:00, 11.95s/it]


Train Loss: 0.0356 | Train Acc: 100.00%
Val Loss: 1.8972 | Val Acc: 41.88%

              precision    recall  f1-score   support

      casual       0.41      0.40      0.41        30
      formal       0.54      0.58      0.56        24
      sporty       0.12      0.10      0.11        10
  streetwear       0.53      0.54      0.53        37
     vintage       0.12      0.12      0.12        16

    accuracy                           0.42       117
   macro avg       0.35      0.35      0.35       117
weighted avg       0.41      0.42      0.41       117



Epoch 20/20: 100%|██████████| 15/15 [02:51<00:00, 11.42s/it]


Train Loss: 0.0247 | Train Acc: 100.00%
Val Loss: 1.9273 | Val Acc: 42.74%

              precision    recall  f1-score   support

      casual       0.42      0.37      0.39        30
      formal       0.48      0.58      0.53        24
      sporty       0.22      0.20      0.21        10
  streetwear       0.54      0.57      0.55        37
     vintage       0.14      0.12      0.13        16

    accuracy                           0.43       117
   macro avg       0.36      0.37      0.36       117
weighted avg       0.42      0.43      0.42       117



In [ ]:
torch.save(model.state_dict(), "efficientnet_fashion_model.pth")

In [ ]:
def predict_image(image_path, model, transform, index_to_label):
    model.eval()
    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(image)
        _, predicted = torch.max(outputs, 1)

    return index_to_label[predicted.item()]


In [ ]:
image_path = "Dataset/images/00892049-fashion_295.jpg"
index_to_label = {i: label for i, label in enumerate(label_encoder.classes_)}

predicted_label = predict_image(image_path, model, val_transform, index_to_label)
print("Predicted style:", predicted_label)

Predicted style: formal
